In [2]:
import cv2
import numpy as np
from pathlib import Path
from scipy import stats
import csv

def read_image(input_path: Path) -> np.ndarray:
    """
    Read the image from the given path using OpenCV.
    """
    img = cv2.imread(str(input_path), cv2.IMREAD_UNCHANGED)
    return img

def save_image(output_path: Path, img: np.ndarray) -> None:
    """
    Save the processed image to the given path using OpenCV.
    """
    cv2.imwrite(str(output_path), img)

def calculate_most_common_color(square: np.ndarray) -> np.ndarray:
    """
    Calculate the most common color of the given square.
    """
    reshaped_square = square.reshape(-1, square.shape[-1])
    mode_color = stats.mode(reshaped_square, axis=0)[0]
    return mode_color

def draw_square(image: np.ndarray, x: int, y: int, square_size: int, border_color: tuple, fill_color: tuple) -> None:
    """
    Draw a square with a specified border and fill color on the image at the specified location.
    """
    image[y:y+square_size, x:x+square_size] = fill_color
    image[y:y+1, x:x+square_size] = border_color
    image[y+square_size-1:y+square_size, x:x+square_size] = border_color
    image[y:y+square_size, x:x+1] = border_color
    image[y:y+square_size, x+square_size-1:x+square_size] = border_color

def process_image(image: np.ndarray, square_size: int) -> np.ndarray:
    """
    Process the image by replacing each square with its most common color and creating a new image with white squares.
    """
    output_image = np.zeros_like(image)
    height, width, _ = image.shape
    color_mapping = []

    for i in range(0, height, square_size):
        for j in range(0, width, square_size):
            square = image[i:i+square_size, j:j+square_size]
            most_common_color = calculate_most_common_color(square)
            color_mapping.append(((j, i), most_common_color.tolist()))

            draw_square(output_image, j, i, square_size, (192, 192, 192, 255), (255, 255, 255, 255))

    return output_image, color_mapping

def write_csv(output_path: Path, color_mapping: list) -> None:
    """
    Write the color mapping to a CSV file.
    """
    with open(output_path, 'w', newline='') as csvfile:
        fieldnames = ['coordinates', 'color']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for coord, color in color_mapping:
            writer.writerow({'coordinates': coord, 'color': color})

def main():
    base_folder = Path("/Users/senthilgandhi/Dropbox/parent/workspace/pixel_puzzle_math")
    input_path = base_folder / "input_image.png"
    output_path = base_folder / "output_image.png"
    csv_path = base_folder / "color_mapping.csv"
    square_size = 16

    input_image = read_image(input_path)
    processed_image, color_mapping = process_image(input_image, square_size)
    save_image(output_path, processed_image)
    write_csv(csv_path, color_mapping)

if __name__ == "__main__":
    main()
